In [1]:
# General imports
import numpy as np
import torch

# DeepMoD stuff
from deepymod_torch import DeepMoD
from deepymod_torch.model.func_approx import NN
from deepymod_torch.model.library import Library1D
from deepymod_torch.model.constraint import LeastSquares
from deepymod_torch.model.sparse_estimators import Threshold
from deepymod_torch.training import train_split_full
from deepymod_torch.training.sparsity_scheduler import TrainTestPeriodic

from phimal_utilities.data import Dataset
from phimal_utilities.data.kdv import DoubleSoliton

if torch.cuda.is_available():
    device ='cuda'
else:
    device = 'cpu'
    
# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

%load_ext autoreload
%autoreload 2

In [6]:
# Making data
v = 0.1
A = 1.0
x = np.linspace(-5, 5, 100)
t = np.linspace(0.0, 1.0, 50)

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(DoubleSoliton, c=[5.0, 2.0], x0=[-3.0, -1.0])
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.4, normalize=False, random=True)
X, y = X.to(device), y.to(device)

In [7]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) #Clustering() # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model
  
sparsity_scheduler = TrainTestPeriodic(patience=8, periodicity=50, delta=1e-5)
optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.999), amsgrad=True, lr=2e-3) # Defining optimizer

In [8]:
train_split_full(model, X, y, optimizer, sparsity_scheduler, log_dir='data/1000_0.2/', write_iterations=25, max_iterations=25000, delta=1e-3, patience=8) # Runni

| Iteration | Progress | Time remaining |     Loss |      MSE |      Reg |    L1 norm |
       3150     12.60%             756s   7.12e-02   7.10e-02   1.85e-04   2.73e+00 Algorithm converged. Stopping training.


In [5]:
model.sparsity_masks

[tensor([False, False, False,  True, False,  True, False, False, False, False,
         False, False], device='cuda:0')]